# Geração de modelos de LDA 

## Corpus: Youtube Influencer_br
## Versão: 3.0


# Método: 

### Para encontrar o melhor k as coerências geradas serão confrontadas com as entropias dos tópicos

### Serão admitidos os seguintes pos_tags: NOUN, PNOUN, ADJ

### Imports

In [1]:
from connections.mongodb_connector import Mongo_Connector
from connections.neo4j_connector import Neo4j_Connector
import os
from datetime import datetime
from gensim import corpora, models, similarities
from models.graph_generator import Graph_Generator
from models.tuple_extractor import Tuple_Extractor
import pickle
from gensim.models.ldamulticore import LdaMulticore
import gensim
from gensim import corpora, models, similarities
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from gensim import corpora, models, similarities
from connections.mysql_connector import MySQL_Connector
from acessos import read, get_conn, persistir_uma_linha, persistir_multiplas_linhas, replace_df
from models.topic_modeling import Topic_Modeling
from gensim.models.wrappers import LdaMallet
import pandas as pd
import operator


#### Carregando Corpus

In [2]:
connector = MySQL_Connector("conn_orfeu")


def buscar_videos_transcripions(conn, idioma, transcription_type="", limit=""):
    transcription_language = "transcription_pt" if idioma =="pt-br" else "transcription_en"
    query = "SELECT  video_id, {} as texto FROM video_transcriptions WHERE transcription_type <> 'Erro'".format(transcription_language)

    if transcription_type != "":
        sub_query = " AND transcription_type = '{}'".format(transcription_type)
        query += sub_query
    
    if limit != "":
        sub_query = " LIMIT {}".format(limit)
        query += sub_query

    df_videos_trans = read(conn, query)
    df_videos_trans.columns=['video_id', "texto"]

    return df_videos_trans

conn = connector.return_conn("influencer_br")

df_videos_trans = buscar_videos_transcripions(conn, "pt-br", limit=10)
df_videos_trans.head()

,video_id,texto
0,--3FrTHN9qw,oi oi gente vídeo de hoje tem pede que o Emoj...
1,--CryguVEik,[Aplausos] [Música] [Aplausos] háháhá 8 a gen...
2,--eP6oj_gkA,contar pra vocês os três principais segredos ...
3,--JOEggYBac,não imaginava que um dia ficar em primeiro lu...
4,--K2l7qd7W0,Vamos falar sobre quais são os alimentos ou s...


### Gerando nome do modelo

In [2]:
date_string = datetime.now().strftime("%d-%m-%Y %H_%M_%S")
path = os.getcwd()
path_modelo = "{}\modelos_lda".format(path)
print(path_modelo)
print(date_string)

D:\Projetos\Mestrado\Projeto PDM\v4\YouGraph\sistema\Python\modelos_lda
09-04-2021 19_12_37


### Parametros do Pré Processamento

In [4]:
    stop_words_list = ["aqui", "gente", "oi", "então", "ai", "aí", "ufa", "coisa", "[Aplausos]", "[Música]", "né","tá", "vídeo",
                       "casar", "coisa","canal", "tipo", "vezes", "mano", "meio", "ea", "eo", "só", "ae", "eis", "caraca", "caramba"
                      ]
allowed_postags = ["NOUN","PROPN", "VERB", "ADJ"]
min_count=10
threshold=50

### Instanciando Obj Topic_Modeling

In [5]:
topic_modeling = Topic_Modeling(language="pt-br",stop_words_list=stop_words_list)

### Gerando lista dos documentos

In [71]:
lista_documentos = df_videos_trans['texto'].tolist()
# lista_documentos[:5]

### Filtrando pos tags

In [72]:
lista_documentos = [topic_modeling.filtrar_pos_tag(texto) for texto in lista_documentos]

## Pré Processamentos

### Tokenizando

In [2]:
lista_documentos_tokenizado = topic_modeling.tokenizar(lista_documentos)
lista_documentos_tokenizado[:5]

## Contanto o número de tokens

In [74]:
def contar_tokens(lista_documentos):
    num_tokens = 0
    #lista_documentos.map(lambda doc: print(type(doc)) )
    for doc in lista_documentos:
        num_tokens = num_tokens + len(doc)
    return num_tokens

num_tokens_sem_processar = contar_tokens(lista_documentos)
 
'Num de tokens: {:,}'.format(num_tokens_sem_processar)

'Num de tokens: 128,868,340'

## Removendo stop words

In [101]:
lista_documentos_tokenizado_stop_words = topic_modeling.remover_stop_words(lista_documentos_tokenizado)

## Gerando BiGramas e TriGramas

In [102]:
lista_documento_bi_gram, lista_documento_tri_gram = topic_modeling.montar_n_grams(lista_documentos_tokenizado_stop_words)

### Encontrando os Bi e Tri Gramas (DE UM JEITO MTO GAMBS)

In [103]:
def encontrar_gramas(lista_documentos_grams):
    lista_grams = []
    for doc in lista_documentos_grams:
        for token in doc:
            if token.find("_") != -1:
                lista_grams.append(token)
    return lista_grams


In [104]:
lista_tri_gramas = encontrar_gramas(lista_documento_tri_gram)
lista_bi_gramas = encontrar_gramas(lista_documento_bi_gram)

In [105]:
#lista_bi_gramas[:50]
lista_tri_gramas[:50]
lista_documento_tri_gram[1][:10]

['aplausos',
 'música',
 'aplausos',
 'háháhá',
 'manias',
 'amor',
 'maior',
 'verdade',
 'muro',
 'anos']

### Lematizando os tokens

In [108]:
lista_documento_lematizada = topic_modeling.lematizar_documentos(lista_documento_tri_gram) 

## Numero de tokens após processamentos

In [129]:
num_tokens_sem_processar = contar_tokens(lista_documentos)
print('Num de tokens: {:,}'.format(num_tokens_sem_processar))

num_tokens_stop_words = contar_tokens(lista_documentos_tokenizado_stop_words)
print('Num de tokens após Stop Words: {:,}'.format(num_tokens_stop_words))


num_tokens_bi_gram  = contar_tokens(lista_documento_bi_gram)
num_tokens_tri_gram = contar_tokens(lista_documento_tri_gram)
print('Num de tokens após com bi_grams: {:,}'.format(num_tokens_bi_gram))
print('Num de tokens após com tri_grams: {:,}'.format(num_tokens_tri_gram))

num_tokens_lematizado = contar_tokens(lista_documento_lematizada)
print("Num de tokens após lematização: {:,}".format(num_tokens_lematizado))

Num de tokens: 128,868,340
Num de tokens após Stop Words: 16,493,979
Num de tokens após com bi_grams: 15,868,240
Num de tokens após com tri_grams: 15,746,434
Num de tokens após lematização: 15,210,559


### Gerando o Id2Word    

In [130]:
id2word = topic_modeling.montar_id2word(lista_documento_lematizada)

### Gerando o Corpus

In [131]:
corpus = topic_modeling.montar_novo_corpus(lista_documento_lematizada, id2word)

Dictionary(153687 unique tokens: ['alice', 'alto', 'alçar', 'amarelar', 'amarelo']...)


### Salvando o ID2WORD e Corpus

In [133]:
dict_corpus = {
    "lista_documentos_tokenizado": lista_documentos_tokenizado,
    "lista_documentos_tokenizado_stop_words": lista_documentos_tokenizado_stop_words,
    "lista_documentos_lematizada":lista_documento_lematizada,
    "lista_documento_tri_gram": lista_documento_tri_gram,
    "lista_tri_gramas": lista_tri_gramas,
    "id2word":id2word,
    "corpus":corpus
    
}

save_path = "{}\\{}".format(path_modelo, date_string)
os.mkdir(save_path) 
with open("{}\\dict_corpus.pickle".format(save_path), 'wb') as handle: #SALVANDO DICT EM UM PICKLE
    pickle.dump(dict_corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)


# Treinamento dos Modelos

### Gerando Modelos

#### Carregando dados

In [3]:
folder_name = "09-11-2020 18_33_23"

path = os.getcwd()
path_modelo = "{}\modelos_lda\{}".format(path, folder_name)

print("-> Lendo Pickle")
with (open("{}\\dict_corpus.pickle".format(path_modelo), "rb")) as openfile:
    while True:
        try:
            dict_corpus = pickle.load(openfile)
        except EOFError:
            break
        

-> Lendo Pickle


In [4]:
corpus = dict_corpus['corpus']
id2word= dict_corpus['id2word']
lista_documentos_tokenizado_stop_words = dict_corpus['lista_documentos_tokenizado_stop_words']
lista_documento_lematizada = dict_corpus['lista_documentos_lematizada']
lista_tri_gramas = dict_corpus["lista_tri_gramas"]

#### Carregando MALLET

In [5]:
mallet_path = "D:\Projetos\Mestrado\mallet-2.0.8"
os.environ['MALLET_HOME'] = mallet_path

mallet_path = "{}\\bin\mallet".format(mallet_path)

mallet_path

'D:\\Projetos\\Mestrado\\mallet-2.0.8\\bin\\mallet'

### Parâmetros

In [13]:
start = 10
limit = 120
step= 10

In [19]:
coherence_values = []
model_list = []
for num_topics in range(start, limit, step):
    print("Gerando novo modelo...")

#     model = LdaMulticore(corpus=corpus, 
#                         id2word=id2word,
#                         random_state=100, 
#                         num_topics=num_topics,
#                         per_word_topics=True,
#                         workers=3)
    
    model = LdaMallet(mallet_path,
                      corpus=corpus, 
                      num_topics=num_topics,
                      workers=2,
                      id2word=id2word)
    
    
    
    print("Novo modelo Gerado..")
    print("Calculando coerencia")
    
    coherencemodel = CoherenceModel(model=model, texts=lista_documento_lematizada, dictionary=id2word, coherence='c_v')
    #coherencemodel = CoherenceModel(model=model, texts=lista_documento_lematizada, dictionary=id2word, coherence='c_npmi')
    
    coherence_values.append(coherencemodel.get_coherence())
    print(" --> Pronto")

    model.save("{}\#_{}".format(path_modelo,num_topics))
    
    model_list.append(model)
    
    #coherence_values = coherence_values
    print("num topics: {}".format(num_topics))
    print("*********")
    print(coherence_values)


Gerando novo modelo...
Novo modelo Gerado..
Calculando coerencia
 --> Pronto
num topics: 10
*********
[0.3921452783749029]
Gerando novo modelo...
Novo modelo Gerado..
Calculando coerencia
 --> Pronto
num topics: 20
*********
[0.3921452783749029, 0.43399610148056356]
Gerando novo modelo...
Novo modelo Gerado..
Calculando coerencia
 --> Pronto
num topics: 30
*********
[0.3921452783749029, 0.43399610148056356, 0.4418800677066677]
Gerando novo modelo...
Novo modelo Gerado..
Calculando coerencia
 --> Pronto
num topics: 40
*********
[0.3921452783749029, 0.43399610148056356, 0.4418800677066677, 0.4536300600784065]
Gerando novo modelo...
Novo modelo Gerado..
Calculando coerencia
 --> Pronto
num topics: 50
*********
[0.3921452783749029, 0.43399610148056356, 0.4418800677066677, 0.4536300600784065, 0.4477756605622723]
Gerando novo modelo...
Novo modelo Gerado..
Calculando coerencia
 --> Pronto
num topics: 60
*********
[0.3921452783749029, 0.43399610148056356, 0.4418800677066677, 0.453630060078406

In [21]:
coherence_values

[0.3921452783749029,
 0.43399610148056356,
 0.4418800677066677,
 0.4536300600784065,
 0.4477756605622723,
 0.46121620611437625,
 0.4593699166129623,
 0.4641071011991242,
 0.4622511691698248,
 0.4561780344894651,
 0.4608962384365008]

### Retornando as top palavras dos tópicos 

In [139]:
modelo_analise = model_list[0]

df_palavras, dict_palavras_topicos = topic_modeling.retornar_top_key_words(modelo_analise, num_palavras=30)

#df_palavras.head(50)
df_palavras.to_csv("09-11-2020 18_33_23 - 50 tópicos.csv")

In [135]:
date_string

'09-11-2020 18_33_23'

In [90]:
# sorted_d = dict( sorted(id2word.cfs.items(), key=operator.itemgetter(1),reverse=True))
# sorted_d

data_contagem = {'id_word': list(id2word.cfs.keys()), 'count': list(id2word.cfs.values())}
df_palavras_count = pd.DataFrame.from_dict(data_contagem)


data_id2token = {'id_word': list(id2word.cfs.keys()), 'word': list(id2word.id2token.values())}
df_id2token = pd.DataFrame.from_dict(data_id2token)



df_palavras_count = pd.merge(df_palavras_count, df_id2token, on='id_word')

df_count_palavras.sort_values(by=['count'], ascending=False).head(50)

,id_word,count,word
280,266,180076,contar
295,357,132506,doença
133,52,123100,perfazer
333,275,121414,matemático
78,26,108945,grande
179,121,106127,álcool_gel
186,185,101435,aparência
367,391,79613,puro
177,118,77616,zinho
345,363,75844,mês


## Entendendo os significados de palavras bizaras (VIDAR, VEZAR, PELAR, MANIR, GATAR, MEAR)

In [1]:
lista_alvo = ["casar"]

for sent in lista_documentos_tokenizado_stop_words:
            doc = topic_modeling.nlp(" ".join(sent))   
            for token in doc :
                lemma = (token.lemma_)
                if lemma in lista_alvo:
                    print("Token {} - Lemma {}".format(token, lemma))
                

In [127]:
lista_documento_lematizada = topic_modeling.remover_stop_words(lista_documento_lematizada)

# Relatório dos Dados

NameError: name 'num_tokens_lematizado' is not defined